In [60]:
%load_ext autoreload
%autoreload 2
import sys 
if '/Users/ericliu/Desktop/Latent-Dirichilet-Allocation' not in sys.path: 
    sys.path.append('/Users/ericliu/Desktop/Latent-Dirichilet-Allocation')

import torch as tr 
import numpy as np 

from src.utils import expec_log_dirichlet, expec_log_dirichlet_mirror

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
%%writefile ../test/test_func_expec_log_dirichlet.py 
import torch as tr 
import numpy as np 

from src.utils import expec_log_dirichlet, expec_log_dirichlet_mirror
tensor1 = tr.tensor([[1,1,1]], dtype=float)
tensor2 = tr.tensor([1,1,1], dtype=float)
tensor3 = tr.tensor([4,5,6], dtype=float)
tensor4 = tr.tensor([[0.9,1,0.2]], dtype=float)
# print(tensor.ndim)
# print(tr.sum(tensor))
for tensor in [tensor1, tensor2, tensor3, tensor4]:
    print(tensor)
    exp_dirichlet_var_ = expec_log_dirichlet(tensor)
    print(exp_dirichlet_var_)

    exp_dirichlet_var = expec_log_dirichlet_mirror(tensor.numpy())
    print(exp_dirichlet_var)
    #print(type(exp_dirichlet_var))
    #print(exp_dirichlet_var.shape)

    assert (
        np.round(exp_dirichlet_var_.numpy(),2) == np.round(exp_dirichlet_var,2)
    ).any(), \
        print(
        np.round(exp_dirichlet_var_.numpy(),2), 
        np.round(exp_dirichlet_var,2)
    )
    print()

Overwriting ../test/test_func_expec_log_dirichlet.py


In [58]:
np.round(exp_dirichlet_var_.numpy(),2)

array([[-1.24, -1.06, -5.77]])

In [55]:
np.round(exp_dirichlet_var,2) 


array([-1.42, -1.17, -0.97])

In [26]:
exp_dirichlet_var.tolist()

[-1.5, -1.5, -1.5]